# Applied Machine Learning Midterm Project: _Classification Analysis_

## Jason Ballard

#### Basehor, Kansas (CDT)

#### April 6, 2025

> Submission: GitHub Repository with Jupyter Notebook and Peer Review
---

## 📌 Project Overview

Organizations frequently need to classify data to support decision-making. 
For example, a healthcare provider may want to predict whether a patient has a specific condition based on lab results,
or a business may classify customer behavior to tailor marketing strategies.
Machine learning classification models help automate these decisions by recognizing patterns in historical data.

This project demonstrates the ability to apply classification modeling techniques to a real-world dataset. You will:

- Load and explore a dataset.
- Analyze feature distributions and consider feature selection.
- Train and evaluate a classification model.
- Compare different classification approaches.
- Document your work in a structured Jupyter Notebook.
- Conduct a peer review of a classmate’s project.
---

## Dependencies
---

In [ ]:
# Standard Library
import logging
import os

# Data Handling
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Utility
import tabulate

# Scikit-learn: Model Selection
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

# Scikit-learn: Models
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Scikit-learn: Preprocessing & Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

# Scikit-learn: Metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

## Section 1. Import and Inspect the Data
---

### - 1.1 Load the dataset and display the first 10 rows.
---

In [ ]:
# Load the Mushroom Classification dataset
column_names = [
    "class", "cap-shape", "cap-surface", "cap-color", "bruises", "odor",
    "gill-attachment", "gill-spacing", "gill-size", "gill-color",
    "stalk-shape", "stalk-root", "stalk-surface-above-ring", "stalk-surface-below-ring",
    "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
    "ring-number", "ring-type", "spore-print-color", "population", "habitat"
]

data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"

df = pd.read_csv(data_url, header=None, names=column_names)

# Display first few rows
print(df.head())


### - 1.2 Check for missing values and display summary statistics.
---

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("🔍 Missing Values:\n", missing_values[missing_values > 0])

# Basic summary statistics (only works meaningfully after encoding)
print("\n📊 Summary Statistics (non-numeric preview):")
print(df.describe(include='all').T)

# Check for class balance
print("\n🎯 Class Distribution:")
print(df['class'].value_counts())

# Visualize missing data if any
if missing_values.any():
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap="YlGnBu")
    plt.title("Missing Data Heatmap")
    plt.show()

## Section 2. Data Exploration and Preparation
---

### - 2.1 Explore data patterns and distributions
  - Create histograms, boxplots, and count plots for categorical variables (as applicable).
  - Identify patterns, outliers, and anomalies in feature distributions.
  - Check for class imbalance in the target variable (as applicable).
  ---

In [ ]:
# Class imbalance check
plt.figure(figsize=(6, 4))
sns.countplot(x='class', data=df, palette='Set2')
plt.title("Class Distribution: Edible vs Poisonous")
plt.xlabel("Class (e = Edible, p = Poisonous)")
plt.ylabel("Count")
plt.show()

# Explore distributions of selected categorical features
categorical_features = ['cap-shape', 'cap-surface', 'cap-color', 'odor', 'gill-color', 'habitat']

# Count plots for each selected feature
for feature in categorical_features:
    plt.figure(figsize=(8, 4))
    sns.countplot(x=feature, data=df, palette='viridis', order=df[feature].value_counts().index)
    plt.title(f"Distribution of {feature}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Check for rare categories or anomalies
print("\n🔎 Unique values per feature:")
for col in df.columns:
    print(f"{col:25} : {df[col].nunique()} unique values")

print("\n Checking for rare categories (less than 10 occurrences):")
for col in df.columns:
    counts = df[col].value_counts()
    rare = counts[counts < 10]


### - 2.2 Handle missing values and clean data
  - Impute or drop missing values (as applicable).
  - Remove or transform outliers (as applicable).
  - Convert categorical data to numerical format using encoding (as applicable).
  ---

In [ ]:
# Re-check missing values before handling
print("🔍 Missing Values Before Cleaning:")
print(df.isnull().sum()[df.isnull().sum() > 0])

# Impute with most frequent value
most_freq = df['stalk-root'].mode()[0]
df['stalk-root'].replace(np.nan, most_freq, inplace=True)

# After imputation, rename to df_clean
df_clean = df.copy()

# Confirm cleaning
print("\n✅ Missing Values After Cleaning:")
print(df_clean.isnull().sum().sum(), "missing values remaining")

# Outlier handling – for categorical data, focus on rare categories
# Drop rare categories or group them into 'Other'

# Encode categorical variables using Label Encoding
# This works well with tree-based models and is simple

df_encoded = df_clean.copy()
label_encoders = {}

for col in df_encoded.columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le  # Save for inverse_transform or decoding later

# Confirm encoding
print("\n🔢 Encoded Data Sample:")
print(df_encoded.head())

### - 2.3 Feature selection and engineering
  - Create new features (as applicable).
  - Transform or combine existing features to improve model performance (as applicable).
  - Scale or normalize data (as applicable).
  ---

In [ ]:
# Separate features and target
X = df_encoded.drop("class", axis=1)
y = df_encoded["class"]

# Optional: Create interaction features (if any domain knowledge suggests it)
# Example (not meaningful here without context):
# X['odor_gill'] = X['odor'] * X['gill-color']  # Not used here, but shown for structure

# Explore feature importance using a simple Decision Tree
# This is a quick way to see which features are most important
# and can help in feature selection
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X, y)

# Get feature importances
importances = pd.Series(dt.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)

# Plot top features
plt.figure(figsize=(10, 6))
sns.barplot(x=importances.values[:10], y=importances.index[:10])
plt.title("Top 10 Important Features (Decision Tree)")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.show()

# Scaling – Required only for distance-based models (e.g., SVM, MLP)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Wrap into DataFrame (optional)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Confirm scaled data
print("\n Scaled Feature Sample:")
print(X_scaled_df.head())


## Section 3. Feature Selection and Justification
---

- 3.1 Choose features and target
  - Select two or more input features (numerical for regression, numerical and/or categorical for classification)
  - Select a target variable (as applicable)
    - Regression: Continuous target variable (e.g., price, temperature).
    - Classification: Categorical target variable (e.g., gender, species).
    - Clustering: No target variable.
  - Justify your selection with reasoning.
---


In [ ]:
# Define features (X) and target (y)
# Target variable: 'class' – edible (e) or poisonous (p)
# Features selected based on earlier Decision Tree feature importance

selected_features = [
    'odor',            # Most influential — strong indicator of toxicity
    'gill-color',      # Indicates maturity and species — useful class separation
    'spore-print-color', # Important taxonomic trait among mushrooms
    'habitat',         # Environmental context — some mushrooms grow in specific regions
    'population'       # Can hint at species commonality
]

# Feature and target definition
X_selected = df_encoded[selected_features]
y = df_encoded['class']  # 0 = edible, 1 = poisonous (after LabelEncoding)

- 3.2 Define X and y
  - Assign input features to X
  - Assign target variable to y (as applicable)
---

In [ ]:
# 3.2 – Define X and y

# Input features (selected from 3.1)
X = df_encoded[[
    'odor',
    'gill-color',
    'spore-print-color',
    'habitat',
    'population'
]]

# Target variable: 'class' (edible vs. poisonous)
y = df_encoded['class']

# Confirm shapes and sample
print(f" Features shape: {X.shape}")
print(f" Target shape: {y.shape}")
print("\n Feature Sample:")
print(X.head())

print("\n Target Sample:")
print(y.head())


## Section 4. Train a Model (Classification: Choose 1: Decision Tree, Random Forest, Logistic Regression)
---

### - 4.1 Split the data into training and test sets using train_test_split (or StratifiedShuffleSplit if class imbalance is an issue).
---

In [ ]:
# Define Stratified Shuffle Split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform the split
for train_index, test_index in split.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Confirm distribution in each split
print(" Class distribution in full dataset:")
print(y.value_counts(normalize=True))

print("\n Class distribution in training set:")
print(y_train.value_counts(normalize=True))

print("\n Class distribution in test set:")
print(y_test.value_counts(normalize=True))

# Confirm shapes
print(f"\n Training set: {X_train.shape}, Test set: {X_test.shape}")


### - 4.2 Train model using Scikit-Learn model.fit() method.
---

In [ ]:
# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model using training data
dt_model.fit(X_train, y_train)

# Confirm training complete
print("✅ Decision Tree model trained successfully.")


In [ ]:


# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Confirm training
print("✅ Random Forest model trained successfully.")


### - 4.3 Evalulate performance, for example:
  - Regression: R^2, MAE, RMSE (RMSE has been recently updated)
  - Classification: Accuracy, Precision, Recall, F1-score, Confusion Matrix
  - Clustering: Inertia, Silhouette Score
---

## Section 5. Improve the Model or Try Alternatives (Implement a Second Option)
---

### - 5.1 Train an alternative classifier (e.g., Decision Tree, Random Forest, Logistic Regression) OR adjust hyperparameters on the original model.
---


### - 5.2 Compare the performance of all models across the same performance metrics.
---

## Section 6. Final Thoughts & Insights
---


### - 6.1 Summarize findings.
---
(write here)    

### - 6.2 Discuss challenges faced.
---
(write here)

### - 6.3 If you had more time, what would you try next?
---
(write here)